# Train First Model: Logistic Regression 

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=JNHO3GLLTYo1IImvEU5Y9l0u5Z7oZo-6u8QJeC-obCo&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/tgG5uez5FLp7JMciT4mT2fE_mLFjVtU6E2_qwQA28XqpCKDmEGbKp7o
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
!gcloud config set project reddit-master

Updated property [core/project].


In [4]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import nltk
import ast
import logging
nltk.download('punkt')

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [6]:
!pwd

/content


In [8]:
%%time

model_LR_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    !gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    print("comments_posts_" + subreddit + ".pkl downloaded")

    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_LR_df = pd.concat([model_LR_df, df], ignore_index=True)

Copying gs://reddit_final_results/comments_posts_aww.pkl...
- [1 files][ 63.8 MiB/ 63.8 MiB]                                                
Operation completed over 1 objects/63.8 MiB.                                     
comments_posts_aww.pkl downloaded


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_nba.pkl...
- [1 files][ 88.3 MiB/ 88.3 MiB]                                                
Operation completed over 1 objects/88.3 MiB.                                     
comments_posts_nba.pkl downloaded


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_movies.pkl...
- [1 files][103.2 MiB/103.2 MiB]                                                
Operation completed over 1 objects/103.2 MiB.                                    
comments_posts_movies.pkl downloaded


...

Copying gs://reddit_final_results/comments_posts_funny.pkl...
\ [1 files][ 69.3 MiB/ 69.3 MiB]                                                
Operation completed over 1 objects/69.3 MiB.                                     
comments_posts_funny.pkl downloaded


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_gaming.pkl...
- [1 files][ 79.4 MiB/ 79.4 MiB]                                                
Operation completed over 1 objects/79.4 MiB.                                     
comments_posts_gaming.pkl downloaded


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


CPU times: user 53 s, sys: 13.2 s, total: 1min 6s
Wall time: 1min 59s


In [9]:
model_LR_df.shape

(10833350, 4)

In [0]:
X = model_LR_df['body']
y = model_LR_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train, 
                X_val=X_val, 
                X_test=X_test, 
                y_train=y_train.astype('int'), 
                y_val=y_val.astype('int'), 
                y_test=y_test.astype('int'))

In [38]:
%time

model_LR_df_split = train_dev_test_split(X,y)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [39]:
model_LR_df_split.keys()

dict_keys(['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test'])

In [40]:
%%time

parameters = [
    {
        'tfidf__max_df': (0.25, 0.5, 0.75),
        'tfidf__ngram_range': [(1,1),(1,2),(1,3)],
        'clf__estimator__class_weight': ["balanced", None]
    }
]

pipeline = Pipeline([
  ('tfidf', TfidfVectorizer(preprocessor=' '.join)),
  ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
])

grid_search_tune = GridSearchCV(pipeline, parameters, cv=3, n_jobs=1, verbose=10)


CPU times: user 334 µs, sys: 38 µs, total: 372 µs
Wall time: 380 µs


In [44]:
y.iloc[5055564]

13

In [0]:
%%time

grid_search_tune.fit(model_LR_df_split['X_train'], model_LR_df_split['y_train'].to_numpy().astype('int'))
            
logging.info('Best parameters set:')
print(grid_search_tune.best_estimator_.steps)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] clf__estimator__class_weight=balanced, tfidf__max_df=0.25, tfidf__ngram_range=(1, 1) 


In [0]:


# measuring performance on test set
logging.info('Applying best classifier on test data:')
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(model_LR_df_split['X_test'])
print(classification_report(model_LR_df_split['y_test'], predictions))

In [0]:
# importing the data - atheism
!gsutil cp gs://reddit_final_results/comments_posts_atheism.pkl .

Copying gs://reddit_final_results/comments_posts_atheism.pkl...
\ [1 files][144.0 MiB/144.0 MiB]                                                
Operation completed over 1 objects/144.0 MiB.                                    


In [0]:
df_atheism = pd.read_pickle("/content/comments_posts_atheism.pkl")

In [0]:
# importing the tfidf vectorizer for atheism 
!gsutil cp gs://reddit_models/content/tfidf_atheism

CommandException: Wrong number of arguments for "cp" command.


In [0]:
# importing the fasttext vectorizer for atheism 
!gsutil cp gs://reddit_models/content/model_fasttext_atheism

CommandException: Wrong number of arguments for "cp" command.


In [0]:
subreddits = list(df_atheism['subreddit'].values)
subreddits

['Unnamed: 0.1', 'subreddit', 'body', 'subreddit_id']

## Train, test and validation - atheism

In [0]:
X_atheism = df_atheism['body']
y_atheism = df_atheism['subreddit_id']

In [0]:
# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train, 
                X_val=X_val, 
                X_test=X_test, 
                y_train=y_train, 
                y_val=y_val, 
                y_test=y_test)

In [0]:
df_atheism_split = train_dev_test_split(X,y)

In [0]:
df_atheism_split.keys()

dict_keys(['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test'])

In [0]:
# Define a pipeline combining a text feature extractor with a Naive Bayes classifier
NB_pipeline = Pipeline([
                ("tfidf", TfidfVectorizer(tokenizer=tokenizer.tokenize)),
                ("clf", OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print(f"Processing {category}...")
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_valid)
    print(f"Test accuracy: {accuracy_score(validation[category], prediction)}")

In [0]:
text_clf = Pipeline([('vectorizer', CountVectorizer()),
    ('transformer', TfidfTransformer()),
    ('classifier', MultinomialNB())])
text_clf.fit(data["train"]["texts"],data["train"]["labels"])
predicted_labels = text_clf.predict(data["test"]["texts"])
print(np.mean(predicted_labels == data["test"]["labels"]))